# Fuzzy Text Matching

### Prerequisities
- Install the Python package `thefuzz` in your favourate package manager

In [1]:
!pip install thefuzz

Defaulting to user installation because normal site-packages is not writeable


Is Vienna the same as Wien, or Vienna, AT, Bécs?

## Applications
- Cities on Github
- Cities on CVs
- Company names
- Public agency names
- Person names

## Reading

In [11]:
from nltk.corpus import words
word_list = words.words()
print(word_list[:10])

['A', 'a', 'aa', 'aal', 'aalii', 'aam', 'Aani', 'aardvark', 'aardwolf', 'Aaron']


In [16]:
len(word_list)

236736

In [19]:
def find_word(word_list, word):
    for candidate in word_list:
        if candidate == word:
            return candidate
    return None

In [23]:
result = find_word(word_list, "whiskye")

In [24]:
print(result)

None


In [25]:
from thefuzz import fuzz

## Distance metric of strings
Take x and y. Distance is 0 if x=y, increasing in some "natural" metric of distance. What Lavenshtein distance does: edit distance. How many characters you have to edit to go from the mistypes version to the correct one?

## Lavenshtein
How many characters do you have to ass/delete/change in x to get y?

`lavenshtein ('whisky', 'whiskey') = 1`
`lavenshtein('whiskey', 'whiskye') = 2`

In [26]:
# the max ratio is 100
fuzz.ratio("whisky", 'whiskey')
# normalizing the distance would be - that's a perfect match

92

In [32]:
# Comparing the total number of characters over the two words
1- 1 / (len('whiskey') +len('whisky'))

0.9230769230769231

In [31]:
fuzz.ratio("app", 'apple')
# 75 is still high, but lower than whisky and whiskey

75

In [33]:
def find_word_fuzzy(word_list, word):
    best_match = None
    # the best lavenhstein score is the lowest
    best_score = 0
    for candidate in word_list:
        score = fuzz.ratio(candidate, word)
        if score > best_score:
            best_match = candidate
            best_score = score
    return best_match

In [44]:
result = find_word_fuzzy(word_list, 'whiskye')
print(result)

whisky


In [48]:
find_word_fuzzy(word_list, 'pálinka')

'lina'

## Resources
- thefuzz
- unidecode

They transliterate, such as removing the accent or changing the Greek or Cirillic letters to latin characters.

In [50]:
%timeit find_word_fuzzy(word_list, 'whiskye')

215 ms ± 37.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


How much is 87 ms? Say 6 million users, comparing against 300k city names. How long will it take?
$$
6 x 10^6 *87 * 10^(3)
$$

$$
145000000 hours --> 6 days
$$

This is unbearably slow.
For $N$ queries among $K$ words, this takes $= N x K$ times.

## Binary search

All variantes complete, on average, in $N\times\log(K)$ times.

## Hash tables
A Python dictionary is a hash table for keys. The key of the dict is converted into a "hash" by a crypto. Lookup by key takes $\approx 1$

In [52]:
N = len(word_list)

In [55]:
# Monatomicity is the middle word
word_list[N//2]

'monatomicity'

In [56]:
# It's at the 25th percentile
word_list[N//4]

'eburnification'

In [58]:
# Hash table
word_dictionary = {k: '' for k in word_list}

In [59]:
# In the dictionary, the word is a key
len(word_dictionary)

235892

In [60]:
%timeit 'whisky' in word_list

4.84 ms ± 613 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [63]:
%timeit 'whisky' in word_dictionary

53.8 ns ± 3.06 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [66]:
letters = 'abcdefghijklopqrstuvwxyz'
words_by_first_letter = dict()
for word in word_list:
    first_letter = word[0].lower()
    if first_letter in words_by_first_letter:
        words_by_first_letter[first_letter].append(word)
    else:
        words_by_first_letter[first_letter] = [word]

In [67]:
len(words_by_first_letter['q'])

1157

In [69]:
words_by_first_letter['q']

['Q',
 'q',
 'qasida',
 'qere',
 'qeri',
 'qintar',
 'Qoheleth',
 'qoph',
 'qua',
 'quab',
 'quabird',
 'quachil',
 'quack',
 'quackery',
 'quackhood',
 'quackish',
 'quackishly',
 'quackishness',
 'quackism',
 'quackle',
 'quacksalver',
 'quackster',
 'quacky',
 'quad',
 'quadded',
 'quaddle',
 'Quader',
 'Quadi',
 'quadmeter',
 'quadra',
 'quadrable',
 'quadragenarian',
 'quadragenarious',
 'Quadragesima',
 'quadragesimal',
 'quadragintesimal',
 'quadral',
 'quadrangle',
 'quadrangled',
 'quadrangular',
 'quadrangularly',
 'quadrangularness',
 'quadrangulate',
 'quadrans',
 'quadrant',
 'quadrantal',
 'quadrantes',
 'Quadrantid',
 'quadrantile',
 'quadrantlike',
 'quadrantly',
 'quadrat',
 'quadrate',
 'quadrated',
 'quadrateness',
 'quadratic',
 'quadratical',
 'quadratically',
 'quadratics',
 'Quadratifera',
 'quadratiferous',
 'quadratojugal',
 'quadratomandibular',
 'quadratosquamosal',
 'quadratrix',
 'quadratum',
 'quadrature',
 'quadratus',
 'quadrauricular',
 'quadrennia',
 '

In [72]:
def find_word_hash(dict_of_words, word):
    first_letter = word[0].lower()
    return find_word_fuzzy(dict_of_words[first_letter], word)

In [74]:
find_word_hash(words_by_first_letter, 'whiskye')

'whisky'

In [76]:
%timeit find_word_fuzzy(word_list, "whiskye")

193 ms ± 39.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [77]:
%timeit find_word_hash(words_by_first_letter, "whiskye")

3.17 ms ± 439 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [78]:
len(words_by_first_letter['w'])

3990